In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import cKDTree
from pyproj import Geod
from shapely.geometry import Point, LineString, Polygon
import cartopy.crs as ccrs
from copy import deepcopy
from obspy.core import UTCDateTime, Trace
from collections import defaultdict
from obspy.core import  read
from obspy.geodetics.base import degrees2kilometers
from matplotlib.pyplot import cm
import matplotlib 
from matplotlib.figure import Figure
import sys, os
import json
import obspy
from obspy import read_inventory
from obspy.core import UTCDateTime
from obspy.core.util import AttribDict
from obspy.core.inventory.util import Equipment
sys.path.append('/g/data/ha3/rakib/seismic/pst/hiperseis')

from seismic.ASDFdatabase.FederatedASDFDataSet import FederatedASDFDataSet

%matplotlib inline

In [2]:
print(obspy.__file__)

/g/data/ha3/Passive/SHARED_ENV/hiperseis_gadi/lib64/python3.6/site-packages/obspy/__init__.py


In [3]:
fds = FederatedASDFDataSet('asdf_files.txt')

Found database: 13274e5d8e6db92411193923a275458638c15f59.db


In [4]:
import glob 
def load_corrections():
    correction_map = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

    pattern = os.path.join(os.path.dirname(fds.asdf_source), '.corrections/*.csv')
    fnames = glob.glob(pattern)

    if(len(fnames)): print('Loading corrections..')

    dtypes = {'net':str, 'sta':str, 'loc':str, 'comp':str, 'date':str,
              'clock_correction':str}
    for fname in fnames:
        df = pd.read_csv(fname, delimiter=',', header=0, dtype=dtypes, na_filter=False)

        try:
            corr_count = 0
            for i in np.arange(len(df)):
                net = df['net'][i]
                sta = df['sta'][i]
                loc = df['loc'][i]
                corr = df['clock_correction'][i]

                if(corr == 'NOXCOR'): continue
                else: corr = float(df['clock_correction'][i])

                st = UTCDateTime(df['date'][i]).timestamp
                et = st + 24*3600

                correction_map[net][sta][loc].append([st, et, corr])
                corr_count += 1
            # end for
        except Exception as e:
            print ('Warning: failed to read corrections file {} with error({}). '
                   'Continuing along..'.format(fname, traceback.format_exc()))
        #end try
    # end for
    
    return correction_map
#end func

In [5]:
clock_corrs = load_corrections()

Loading corrections..


In [6]:
import json

def load_orientations_corrections(path_to_files):
    rf_correction_map = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    swp_correction_map = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

    pattern = os.path.join(path_to_files, '*.json')
    fnames = glob.glob(pattern)

    if(len(fnames)): print('Loading corrections..')

    for fname in fnames:
        orientations = json.load(open(fname))

        try:
            rf_dict = orientations['rf']
            swp_dict = orientations['swp']

            for k,v in rf_dict.items():
                try:
                    net, sta, loc = k.split('.')
                    st, et = v['date_range']
                    az_corr = v['azimuth_correction']
                    rf_correction_map[net][sta][loc].append([st, et, az_corr])
                except:
                    print('No RF correction found for station {}'.format(k))
                # end if
            # end for
            
            for k,v in swp_dict.items():
                try:
                    net, sta, loc = k.split('.')
                    st, et = v['date_range']
                    az_corr = v['azimuth_correction']
                    az_uncert = v['uncertainty']
                    swp_correction_map[net][sta][loc].append([st, et, az_corr, az_uncert])
                except:
                    print('No SWP correction found for station {}'.format(k))
                # end if                    
            # end for            
        except Exception as e:
            print ('Warning: failed to read corrections file {} with error({}). '
                   'Continuing along..'.format(fname, traceback.format_exc()))
        #end try
    # end for
    
    return rf_correction_map, swp_correction_map
# end func

rf_orientations, swp_orientations = \
load_orientations_corrections('orientations/')

Loading corrections..
No RF correction found for station OA.BS24.


In [7]:
# get response object
resp_file = '/g/data/ha3/Passive/SHARED_DATA/Inventory/Station_Extra_Metadata/Equipments/RESP.COMPACT120S.MINIMUS.txt'
resp_inv = obspy.read_inventory(resp_file, format='RESP')

datetime = UTCDateTime("2015-01-01T00:00:00")
resp_obj = resp_inv.get_response("XX.XX..BHZ", datetime)
print(resp_obj)

Channel Response
	From M/S (Velocity in Meters per Second) to COUNTS (Digital Counts)
	Overall Sensitivity: 3.09063e+08 defined at 1.000 Hz
	2 stages:
		Stage 1: PolesZerosResponseStage from M/S to V, gain: 754.3
		Stage 2: CoefficientsTypeResponseStage from V to COUNTS, gain: 409735


In [8]:
# get inventory from fds and create a merged inventory with responses and corrections
inv = fds.get_inventory(network='OA')

# keep stations are until 2019-12-31 (removed CA45 and BQ42 from sqlite output)
nslc_list = open('stations_to_2019_12_31.txt').readlines()
nslc_list = set([item.strip() for item in nslc_list])

ns_set = set()
for nslc in nslc_list:
    net,sta,loc,cha = nslc.split('.')
    ns_set.add('{}.{}'.format(net, sta))
# end for

oinv = None
for ns in ns_set:
    net,sta = ns.split('.')
    if(oinv is None): oinv = inv.select(network=net, station=sta).copy()
    else: 
        cinv = inv.select(network=net, station=sta).copy()
        for net in cinv.networks:
            for sta in net.stations:
                oinv.networks[0].stations.append(sta)
                if(oinv.networks[0].start_date > sta.start_date): 
                    oinv.networks[0].start_date = sta.start_date
                if(oinv.networks[0].end_date < sta.end_date): 
                    oinv.networks[0].end_date = sta.end_date
# end for

# add responses and equipment
for net in oinv.networks:
    for sta in net.stations:
        for cha in sta.channels:
            cha.response = resp_obj
            
            sensor = Equipment(type='Sensor', 
                               description='Nanometrics Trillium Compact 120s')
            digitizer = Equipment(type='Digitizer', 
                               description='Guralp Minimus')
            
            cha.equipments = []
            cha.equipments.append(sensor) 
            cha.equipments.append(digitizer) 
        # end for
    # end for
# end for

# define namespace
fmt="%Y-%m-%dT%H:%M:%S"
nsmap = {}
nsmap['GeoscienceAustralia']= 'https://github.com/GeoscienceAustralia/hiperseis'
# add corrections
for net in oinv.networks:
    for sta in net.stations:  
        sta.extra = AttribDict()

        # add gps clock corrections
        if(len(clock_corrs[net.code][sta.code])):
            ad = AttribDict()
            ad.namespace = nsmap['GeoscienceAustralia']
            
            corr_dict = {}
            for lk in clock_corrs[net.code][sta.code].keys():
                corr_list = []
                for row in clock_corrs[net.code][sta.code][lk]:
                    st, et, corr = row
                    key = '{} - {}'.format(UTCDateTime(st).strftime(fmt),
                                           UTCDateTime(et).strftime(fmt))
                    corr_list.append({key: corr})
                # end for
                corr_dict[lk] = corr_list
            # end for
            
            ad.value = json.dumps(corr_dict)
            sta.extra.clock_corrections = ad
        # end if        
        
        # add rf orientation corrections
        if(len(rf_orientations[net.code][sta.code])):
            ad = AttribDict()
            ad.namespace = nsmap['GeoscienceAustralia']
            
            corr_dict = {}
            for lk in rf_orientations[net.code][sta.code].keys():
                st, et, corr = rf_orientations[net.code][sta.code][lk][0]
                key = '{} - {}'.format(UTCDateTime(st).strftime(fmt),
                                       UTCDateTime(et).strftime(fmt))
                corr_dict[lk] = {key: corr}
            # end for
            
            ad.value = json.dumps(corr_dict)
            sta.extra.rf_orientation_corrections = ad
        # end if                
        
        # add swp orientation corrections
        if(len(swp_orientations[net.code][sta.code])):
            ad = AttribDict()
            ad.namespace = nsmap['GeoscienceAustralia']
            
            corr_dict = {}
            for lk in swp_orientations[net.code][sta.code].keys():
                st, et, corr, corr_uncert = swp_orientations[net.code][sta.code][lk][0]
                key = '{} - {}'.format(UTCDateTime(st).strftime(fmt),
                                       UTCDateTime(et).strftime(fmt))
                corr_dict[lk] = {key: [round(corr, 1), round(corr_uncert, 1)]}
            # end for
            
            ad.value = json.dumps(corr_dict)
            sta.extra.swp_orientation_corrections = ad
        # end if                        
    # end for
# end for

#print(oinv)

In [9]:
oinv.write('OA.xml', format='STATIONXML', nsmap=nsmap)

In [10]:
# plot map of stations

netstas = []
lons = [] 
lats = []

for net in oinv.networks:
    for sta in net.stations:
        netstas.append('{}.{}'.format(net.code, sta.code))
        lons.append(sta.longitude)
        lats.append(sta.latitude)
    # end for
# end for

lons = np.array(lons)
lats = np.array(lats)

In [11]:
clon = np.mean(lons)
crs = ccrs.PlateCarree(central_longitude=clon)

left = 0.05
bottom = 0.05
width = 0.9
height = 0.8

fig = Figure(figsize=(12, 5))
ax = fig.add_axes([left, bottom, width, height], projection=crs)


# draw coastlines.
ax.coastlines('50m')
gl = ax.gridlines(crs=crs, draw_labels=True,
                  dms=False,
                  linewidth=1, color='gray',
                  alpha=0.5, linestyle='--')
# plot stations
for i, sc in enumerate(netstas):
    lon, lat = lons[i], lats[i]

    px, py = lon, lat
    pxl, pyl = lon + 0.02, lat - 0.1
    ax.scatter(px, py, 20, transform=crs, marker='v', c='r', edgecolor='none', zorder=1)
    ax.annotate(sc.split('.')[1], xy=(pxl, pyl), fontsize=5, zorder=2)
# end for
fig.savefig('OA_stations.pdf')

/g/data/ha3/Passive/SHARED_ENV/hiperseis_gadi/lib64/python3.6/site-packages/cartopy/crs.py:228: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
/g/data/ha3/Passive/SHARED_ENV/hiperseis_gadi/lib64/python3.6/site-packages/cartopy/crs.py:280: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
/g/data/ha3/Passive/SHARED_ENV/hiperseis_gadi/lib64/python3.6/site-packages/cartopy/crs.py:347: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0